In [28]:
import hvplot.pandas
import pandas as pd
import warnings
from config import geoapify_key
import requests

In [2]:
warnings.filterwarnings("ignore")
df = pd.read_csv("weather_data/cities.csv")
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,42.85,99,11,1.19,NZ,2023-01-12 15:25:25
1,1,Punta Arenas,-53.1500,-70.9167,59.11,58,75,28.77,CL,2023-01-12 15:23:11
2,2,Castro,-24.7911,-50.0119,74.37,70,100,4.92,BR,2023-01-12 15:25:25
3,3,Mayo,38.8876,-76.5119,49.06,89,0,5.75,US,2023-01-12 15:25:26
4,4,Nabire,-3.3667,135.4833,78.21,88,100,2.37,ID,2023-01-12 15:25:09


In [3]:
df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [6]:
map_plot_1 = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height= 500
)
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

In [9]:
# Configure the map plot_3
map_plot_3 = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.8,
    color = "City"
)

# Display the map plot
map_plot_3

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [11]:
# Configure the map plot_4
map_plot_4 = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

In [14]:
# Configure the map plot_5 where size is based on wind speed
map_plot_5 = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Wind Speed",
    scale = 1.0,
    color = "City"
)

# Display the map plot
map_plot_5

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Wind Speed)

In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [18]:
preferred_cities_df = df.loc[
    (df["Max Temp"] <= max_temp) & (df["Max Temp"] >= min_temp)
]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Castro,-24.7911,-50.0119,74.37,70,100,4.92,BR,2023-01-12 15:25:25
4,4,Nabire,-3.3667,135.4833,78.21,88,100,2.37,ID,2023-01-12 15:25:09
5,5,Marzuq,14.4000,46.4667,69.60,49,55,3.53,YE,2023-01-12 15:25:27
6,6,Sibolga,1.7427,98.7792,75.81,90,83,3.02,ID,2023-01-12 15:25:27
7,7,Tautira,-17.7333,-149.1500,76.46,82,100,24.83,PF,2023-01-12 15:25:29
8,8,Vaini,-21.2000,-175.2000,78.96,94,75,9.22,TO,2023-01-12 15:25:31
9,9,Isangel,-19.5500,169.2667,76.41,81,48,5.91,VU,2023-01-12 15:25:32
12,12,Hobart,-42.8794,147.3294,61.27,87,40,4.61,AU,2023-01-12 15:25:34
13,13,Alta Floresta,-9.8756,-56.0861,84.54,66,85,1.61,BR,2023-01-12 15:25:34
15,15,Mercedes,-34.6515,-59.4307,86.58,20,22,7.94,AR,2023-01-12 15:25:35


In [19]:
preferred_cities_df.count()

City_ID       281
City          281
Lat           281
Lng           281
Max Temp      281
Humidity      281
Cloudiness    281
Wind Speed    281
Country       280
Date          281
dtype: int64

In [20]:
preferred_cities_df.dropna(inplace=True)

In [22]:
preferred_cities_df.count()

City_ID       280
City          280
Lat           280
Lng           280
Max Temp      280
Humidity      280
Cloudiness    280
Wind Speed    280
Country       280
Date          280
dtype: int64

In [23]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Castro,BR,74.37,-24.7911,-50.0119,
4,Nabire,ID,78.21,-3.3667,135.4833,
5,Marzuq,YE,69.60,14.4000,46.4667,
6,Sibolga,ID,75.81,1.7427,98.7792,
7,Tautira,PF,76.46,-17.7333,-149.1500,
8,Vaini,TO,78.96,-21.2000,-175.2000,
9,Isangel,VU,76.41,-19.5500,169.2667,
12,Hobart,AU,61.27,-42.8794,147.3294,
13,Alta Floresta,BR,84.54,-9.8756,-56.0861,
15,Mercedes,AR,86.58,-34.6515,-59.4307,


In [26]:
# Set initial parameters to search for a hotel.
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

In [31]:
# Set the parameters to search for a hotel in Paris.
latitude = 48.8566
longitude = 2.3522
radius = 5000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "filter": f"circle:{longitude},{latitude},{radius}"
}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    try:
        # Get the first hotel from the results and store the name
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except(KeyError,IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

In [32]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Castro,BR,74.37,-24.7911,-50.0119,Buganville Express
4,Nabire,ID,78.21,-3.3667,135.4833,Hotel Anggrek
5,Marzuq,YE,69.60,14.4000,46.4667,No hotel found
6,Sibolga,ID,75.81,1.7427,98.7792,Hotel Hotma
7,Tautira,PF,76.46,-17.7333,-149.1500,No hotel found
...,...,...,...,...,...,...
545,Honiara,SB,76.50,-9.4333,159.9500,Solomon Kitano Mendana Hotel
546,Tahta,EG,60.98,26.7693,31.5021,No hotel found
547,Avera,US,65.55,33.1940,-82.5271,No hotel found
548,Anastacio,BR,86.54,-20.4836,-55.8069,Fênix Plaza Hotel


In [36]:
# Configure the map
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = .5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)